In [132]:
import pandas as pd
import numpy as np
from pathlib import Path
from src.metrics.metrics import Metrics
from src.data.data_utils import stratified_split_multidim_kmeans, stratified_split_multidim_proj_1d
# model
import xgboost as xgb
from src.models.my_models.kerasModels.dnn import DNN, DNN_th
from sklearn.model_selection import train_test_split


#  data transformations
from src.data.data_utils import augment_data, fill_value_2d, transform_x

In [133]:
# Params
path_dataset_world = Path("../../data/datasets/hplc_world/")
path_dataset_med  = Path("../../data/datasets/dataset_hplc_multi")

vars_13 =       ['400', '412', '442', '490', '510', '560', '620', '665', '673', '681', '708', '778', '865', 'lat', 'lon']
legacy_5_vars = [       '412', '442', '490',        '560',               '673',                             'lat', 'lon']
missing_vars =   ['400',                      '510',        '620', '665',        '681', '708', '778', '865']

pigments = ['chlide_a[mg*m^3]', 'chla[mg*m^3]', 'chlb[mg*m^3]', 'chlc1+c2[mg*m^3]',
       'fucox[mg*m^3]', "19'hxfcx[mg*m^3]", "19'btfcx[mg*m^3]",
       'diadino[mg*m^3]', 'allox[mg*m^3]', 'diatox[mg*m^3]', 'zeaxan[mg*m^3]',
       'beta_car[mg*m^3]', 'peridinin[mg*m^3]']
mets = Metrics()

In [134]:
x_wor = pd.read_csv(path_dataset_world / 'rrs_world.csv', index_col=0)
y_wor = pd.read_csv(path_dataset_world / 'hplc_world.csv', index_col=0)

x_med = pd.read_csv(path_dataset_med / 'log_rrs_lat_lon_month_season_depth_loc.csv').drop(columns=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', 'spring', 'winter', 'autumn', 'summer'])
y_med = pd.read_csv(path_dataset_med / 'log_pigments.csv')

In [135]:
x_wor = x_wor[legacy_5_vars]
y_wor = y_wor[pigments]
x_wor_13 = x_wor.copy()
x_wor_13[missing_vars] = np.nan
x_wor_13 = x_wor_13[vars_13]

x_med_13 = x_med[vars_13]
x_med = x_med[legacy_5_vars]
y_med = y_med[pigments]

In [136]:

def p1d(y):
    # return y.sum(axis=1)
    return y['zeaxan[mg*m^3]']
# split = stratified_split_multidim_proj_1d(x_med, y_med, p1d, q=20, test_size=0.2, random_state=42)

# split = stratified_split_multidim_kmeans(x_med, y_med, clusters=10, test_size=0.2, random_state=1)


In [137]:
r = 5
split = stratified_split_multidim_kmeans(x_med, y_med, clusters=10, test_size=0.1, random_state=r)

for id_train, id_test in split:
    x_med_train, x_med_test = x_med.loc[id_train, :].copy(), x_med.loc[id_test, :].copy()
    y_med_train, y_med_test = y_med.loc[id_train, :].copy(), y_med.loc[id_test, :].copy()
    x_med_train_13, x_med_test_13 = x_med_13.loc[id_train, :].copy(), x_med_13.loc[id_test, :].copy()
    y_med_train_13, y_med_test_13 = y_med.loc[id_train, :].copy(), y_med.loc[id_test, :].copy()

# x_med_train, x_med_test,  y_med_train, y_med_test  = train_test_split(x_med, y_med, test_size=0.1, random_state=r)
# x_med_train_13, x_med_test_13,  y_med_train_13, y_med_test_13  = train_test_split(x_med_13, y_med, test_size=0.1, random_state=r)

x_med_train_aug, y_med_train_aug = augment_data(x_med_train, y_med_train, replicate=1000)
x_med_train_13_aug, y_med_train_13_aug = augment_data(x_med_train_13, y_med_train_13, replicate=1000)


In [138]:
# split = stratified_split_multidim_kmeans(x_wor, y_wor, clusters=10, test_size=0.2, random_state=1)

# for id_train, id_test in split:
#     x_wor_train, x_wor_test = x_wor.loc[id_train, :], x_wor.loc[id_test, :]
#     y_wor_train, y_wor_test = y_wor.loc[id_train, :], y_wor.loc[id_test, :]

x_wor_train, x_wor_test, y_wor_train, y_wor_test = train_test_split(x_wor, y_wor, random_state=42, test_size=0.1)
m = y_wor_train.mean()
y_wor_train = y_wor_train.fillna(m)
x_wor_train_aug, y_wor_train_aug = augment_data(x_wor_train, y_wor_train, replicate=10)


x_wor_train_13, x_wor_test_13, y_wor_train_13, y_wor_test_13 = train_test_split(x_wor_13, y_wor, random_state=42, test_size=0.1)
m = y_wor_train_13.mean()
y_wor_train_13 = y_wor_train_13.fillna(m)
x_wor_train_13_aug, y_wor_train_13_aug = augment_data(x_wor_train_13, y_wor_train_13, replicate=10)

In [139]:
x_wor_train_13_aug

,400,412,442,490,510,560,620,665,673,681,708,778,865,lat,lon
0,NaN,-5.956982,-6.098545,-5.944607,NaN,-6.004469,NaN,NaN,-7.262660,NaN,NaN,NaN,NaN,40.854168,-70.395836
1,NaN,-4.335204,-4.642688,-5.095384,NaN,-6.556578,NaN,NaN,-9.389499,NaN,NaN,NaN,NaN,21.020830,-157.937500
2,NaN,-5.106430,-5.194128,-5.401149,NaN,-6.558749,NaN,NaN,-8.536194,NaN,NaN,NaN,NaN,0.020831,-9.895828
3,NaN,-4.154369,-4.444517,-4.875556,NaN,-6.125488,NaN,NaN,-7.991570,NaN,NaN,NaN,NaN,20.604166,-157.062500
4,NaN,-8.501501,-8.289497,-7.173455,NaN,-6.117101,NaN,NaN,-10.433021,NaN,NaN,NaN,NaN,40.895832,-73.729164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75565,NaN,-5.286927,-4.999339,-4.792618,NaN,-5.073350,NaN,NaN,-9.004359,NaN,NaN,NaN,NaN,26.907060,-81.414046
75566,NaN,-5.296452,-5.009408,-4.882597,NaN,-5.219268,NaN,NaN,-8.833931,NaN,NaN,NaN,NaN,27.031459,-85.295313
75567,NaN,-5.462265,-4.910234,-4.791758,NaN,-5.224483,NaN,NaN,-9.207565,NaN,NaN,NaN,NaN,27.372511,-89.218403
75568,NaN,-5.508552,-5.207116,-4.734226,NaN,-5.071381,NaN,NaN,-9.442178,NaN,NaN,NaN,NaN,28.992137,-82.306392


In [115]:
xgb_world = xgb.XGBRegressor(
    n_estimators=225,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0,
    reg_alpha=0,
    reg_lambda=1,
    objective='reg:squarederror',
    booster='gbtree',
    tree_method='auto'
)
xgb_med = xgb.XGBRegressor(
    n_estimators=225,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0,
    reg_alpha=0,
    reg_lambda=1,
    objective='reg:squarederror',
    booster='gbtree',
    tree_method='auto'
)
xgb_merge = xgb.XGBRegressor(
    n_estimators=225,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0,
    reg_alpha=0,
    reg_lambda=1,
    objective='reg:squarederror',
    booster='gbtree',
    tree_method='auto'
)

xgb_med_13 = xgb.XGBRegressor(
    n_estimators=225,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0,
    reg_alpha=0,
    reg_lambda=1,
    objective='reg:squarederror',
    booster='gbtree',
    tree_method='auto'
)
xgb_merge_13 = xgb.XGBRegressor(
    n_estimators=225,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0,
    reg_alpha=0,
    reg_lambda=1,
    objective='reg:squarederror',
    booster='gbtree',
    tree_method='auto'
)
# xgb_world = xgb.XGBRegressor()
# xgb_med = xgb.XGBRegressor()
# xgb_merge = xgb.XGBRegressor()

In [116]:
xgb_world.fit(x_wor_train_aug, y_wor_train_aug)


XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=225, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [117]:
xgb_med.fit(x_med_train_aug, y_med_train_aug)


XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=225, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [118]:
xgb_merge.fit(pd.concat([x_wor_train_aug, x_med_train_aug]), pd.concat([y_wor_train_aug, y_med_train_aug]))

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=225, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [119]:
xgb_merge_13.fit(pd.concat([x_wor_train_13_aug, x_med_train_13_aug]), pd.concat([y_wor_train_13_aug, y_med_train_13_aug]))

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=225, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [120]:
xgb_med_13.fit(x_med_train_13_aug, y_med_train_13_aug)


XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=225, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [126]:
py = xgb_merge.predict(x_med_test)

m = mets.compute_metrics(y_med_test.values, py)
pd.DataFrame(m, index=pigments, columns=mets.metrics.keys()).mean()

R2      0.682042
MAPE    0.773577
MAE     0.039696
MSE     0.008817
dtype: float64

In [127]:
py = xgb_merge_13.predict(x_med_test_13)

m = mets.compute_metrics(y_med_test.values, py)
pd.DataFrame(m, index=pigments, columns=mets.metrics.keys()).mean()

R2      0.685632
MAPE    1.273464
MAE     0.034695
MSE     0.007718
dtype: float64

In [128]:
py = xgb_med.predict(x_med_test)

m = mets.compute_metrics(y_med_test.values, py)
pd.DataFrame(m, index=pigments, columns=mets.metrics.keys()).mean()

R2      0.690962
MAPE    0.837657
MAE     0.039775
MSE     0.009112
dtype: float64

In [129]:
py = xgb_med_13.predict(x_med_test_13)

m = mets.compute_metrics(y_med_test.values, py)
pd.DataFrame(m, index=pigments, columns=mets.metrics.keys()).mean()

R2      0.676376
MAPE    1.609381
MAE     0.035672
MSE     0.007972
dtype: float64

In [130]:
py = xgb_merge.predict(x_med_train)

m = mets.compute_metrics(y_med_train.values, py)
pd.DataFrame(m, index=pigments, columns=mets.metrics.keys()).mean()

R2      0.949976
MAPE    0.210291
MAE     0.026172
MSE     0.013848
dtype: float64

In [131]:
py = xgb_merge_13.predict(x_med_train_13)

m = mets.compute_metrics(y_med_train.values, py)
pd.DataFrame(m, index=pigments, columns=mets.metrics.keys()).mean()

R2      0.971103
MAPE    0.151832
MAE     0.017996
MSE     0.008001
dtype: float64

In [80]:
py = xgb_med.predict(x_med_train)

m = mets.compute_metrics(y_med_train.values, py)
pd.DataFrame(m, index=pigments, columns=mets.metrics.keys()).mean()

R2      0.962630
MAPE    0.181479
MAE     0.024741
MSE     0.014637
dtype: float64

In [81]:
py = xgb_med_13.predict(x_med_train_13)

m = mets.compute_metrics(y_med_train_13.values, py)
pd.DataFrame(m, index=pigments, columns=mets.metrics.keys()).mean()

R2      0.980057
MAPE    0.126532
MAE     0.016216
MSE     0.007923
dtype: float64

In [82]:
py = xgb_med_13.predict(x_med_train_13)

m = mets.compute_metrics(y_med_train_13.values, py)
pd.DataFrame(m, index=pigments, columns=mets.metrics.keys()).mean()

R2      0.980057
MAPE    0.126532
MAE     0.016216
MSE     0.007923
dtype: float64

In [ ]:
def ncv_xgboost(x, y, param_d, score, score_aux=None, outer_splits=5, inner_splits=3, seed=1, n_it=50):
    kfold_outer = KFold(n_splits=outer_splits, shuffle=True, random_state=seed)
    train_result = {}
    test_result = {}
    i = 0    
    for train_idx, test_idx in tqdm(kfold_outer.split(x), total=outer_splits):
        X_train, X_test = x.iloc[train_idx].copy(), x.iloc[test_idx].copy()
        y_train, y_test = y.iloc[train_idx].copy(), y.iloc[test_idx].copy()
        X_train, y_train = augment_data(X_train, y_train, replicate=1)
        # X_train.loc[:, ['lat']] = fill_value_2d(X_train.loc[:, ['lat']].values, val= np.nan, percent=0.1)
        # X_train.loc[:, ['lon']] = fill_value_2d(X_train.loc[:, ['lon']].values, val= np.nan, percent=0.1)
        #X_train.loc[:, :] = fill_random_2d(X_train.loc[:, :].values, 0.1)
        m = y_train.mean()
        y_train = y_train.fillna(m)
        
        kfold_inner = KFold(n_splits=inner_splits, shuffle=True, random_state=seed)
        
        # RandomizedSearchCV for hyperparameter tuning
        model = xgb.XGBRegressor(random_state=seed, verbosity=0)
        randomized_search = RandomizedSearchCV(
            estimator=model,
            param_distributions=param_d,
            n_iter=n_it, 
            scoring=score,
            cv=kfold_inner,
            # cv=inner_splits,
            random_state=seed,
            n_jobs=-1
        )

        # Fit RandomizedSearchCV
        randomized_search.fit(X_train, y_train)
    
        # Get the best hyperparameters
        best_params = randomized_search.best_params_
    
        # Train a new model on the entire outer training set using the best hyperparameters
        best_model = xgb.XGBRegressor(random_state=seed, verbosity=0, **best_params)
        best_model.fit(X_train, y_train)
    
        # Evaluate the model on the outer test set
        aux_res_train = {}
        py = best_model.predict(X_train)
        py = pd.DataFrame(py, columns=y_train.columns)
        if score_aux is not None:
            aux_res_train = {'aux_mets': aux_mets(y_train, py)}
        train_result[i] = {**best_params, "score": score(best_model, X_train, y_train), **aux_res_train, "mean decrease impurity": best_model.feature_importances_}
    
        # Evaluate the model on the outer test set
        aux_res_test = {}
        py = best_model.predict(X_test)
        py = pd.DataFrame(py, columns=y_test.columns)

        if score_aux is not None:
            aux_res_test = {'aux_mets': aux_mets(y_test, py)}
        pi = permutation_importance(best_model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2, scoring=score)
        test_result[i] = { **best_params, "score": score(best_model, X_test, y_test), **aux_res_test, 
                    "permutation importance mean": pi.importances_mean,
                    "permutation importance std": pi.importances_std,}
        i = i+1
    return train_result, test_result